In [1]:
import pandas as pd
import numpy as np
import os
import xlsxwriter

In [2]:
SURVEYS_CORE_RESPONSES_FILE = "Survey_Responses_Comments.xlsx"
CORE_RESPONSES_TO_SKIP = 172
SURVEYS_PEOPLE_DETAILS_FILES = ["responses1.xlsx",'responses2.xlsx','responses_3rd survey.xlsx']
OUTPUT_FILE_HEADERS = ["Email","Name","Affiliation","Designation","source"]

In [3]:
CORE_RESPONSES = pd.read_excel(SURVEYS_CORE_RESPONSES_FILE)
CORE_RESPONSES.head()

,Timestamp,Comment Number,Comment Classification,Comment Views,Alternate Comment,Email Id,Full Name,Designation,Affiliation
0,2019-12-20 22:57:59.045,1:comment1,U,IDK,NOPE,my@gmail.com,mynam,stud,skul
1,2019-12-20 23:38:30.209,index:comment1,PU,oijo,jihh,abc@gmail.com,fdjs hjh,oh u,huh uhu
2,2019-12-20 23:40:03.682,curl_util:comment6,NU,IDK,NOPE,test@gmail.com,TEST,TESTDE,TESTAFF
3,2019-12-20 23:53:13.494,plplot_plmem:comment3,U,NO,YES,TEST@TESTMAIL.COM,TEST2,TT,AFF
4,2019-12-20 23:56:44.150,libpng_pngmem:comment4,PU,hh,kk,TEST@TT.com,TEST3,TT,TT


In [4]:
CORE_RESPONSES = np.array(CORE_RESPONSES)[CORE_RESPONSES_TO_SKIP:]

In [5]:
'''
Read all comments core responses (taken from HTML Webpage) in a map. 
Key is file name value is another map whose key is comment number and value is a list of all responses.
'''
CORE_RESPONSES_MAP = {}
for res in CORE_RESPONSES:
    filename, commentId = res[1].split(":")[0],res[1].split(":")[1]
    if filename not in CORE_RESPONSES_MAP:
        CORE_RESPONSES_MAP[filename] = {}
    if commentId not in CORE_RESPONSES_MAP[filename]:
        CORE_RESPONSES_MAP[filename][commentId] = []
    CORE_RESPONSES_MAP[filename][commentId].append(res)
#CORE_RESPONSES_MAP

In [6]:
'''
Map containg details of all people (taken from Typeform).
Key is email id and value is details as filled in typeform.
'''
PEOPLE_DETAILS = {}
for file in SURVEYS_PEOPLE_DETAILS_FILES:
    data = pd.read_excel(file)
    data = np.array(data)
    for el in data:
        email = el[6]
        if email in PEOPLE_DETAILS:
            print("Repeating Email Id:",email)
            print(PEOPLE_DETAILS[email])
            print(el)
            print("--------------------------------------------------------")
        PEOPLE_DETAILS[email] = el

Repeating Email Id: karjun312@gmail.com
['60ewgnqssmg534jjghf60ewgng81h41h' 'Amit Kumar' 'IIT kharagpur' 'CSE'
 '18CS60R44' 'Mtech Student' 'karjun312@gmail.com' 'Done' 'Done' 'No'
 'Yes' 'No' 'xxxxx' Timestamp('2020-01-21 06:04:53')
 Timestamp('2020-01-21 06:23:30') '4fd1b43ea3']
['qstjjwdy7gvih5nuxwaqstjjwdtxvu4y' 'Amit Kumar' 'IIT kharagpur' 'CSE'
 '18CS60R44' 'M.Tech 2nd year student' 'karjun312@gmail.com' 'Done' 'Done'
 'No' 'Yes, It is interesting.' 'No' 'xxxxx'
 Timestamp('2020-01-21 05:58:44') Timestamp('2020-01-21 06:03:32')
 '4fd1b43ea3']
--------------------------------------------------------
Repeating Email Id: dhruvkayastha@iitkgp.ac.in
['jzaetljygsn5im9wjfoojzaet07702hj' 'Dhruv' 'IIT KGP' nan nan 'Student'
 'dhruvkayastha@iitkgp.ac.in' 'done' 'done' 'no' 'ok' nan 'xxxxx'
 Timestamp('2020-01-19 10:17:12') Timestamp('2020-01-19 10:21:37')
 'f165b88ce9']
['nw2aakhzjwe5ulrfnw2aav3n6qorm5zn' 'Dhruv' 'IIT KGP' nan nan 'Student'
 'dhruvkayastha@iitkgp.ac.in' 'done' 'done' 'no' 

In [8]:
'''
Merge people's details from Typeform and comments responses and make an excel.
'''
if not os.path.exists("output"):
    os.mkdir("output")
for filename in CORE_RESPONSES_MAP:
    file_map = CORE_RESPONSES_MAP[filename]
    num_comments = len(file_map)
    workbook = xlsxwriter.Workbook('output/'+filename + '.xlsx')
    ws = workbook.add_worksheet()
    for j,header in enumerate(OUTPUT_FILE_HEADERS):
        ws.write(0,j,header)
    j = 0
    comment_key_to_index = {}
    for k in file_map.keys():
        ws.merge_range(0,len(OUTPUT_FILE_HEADERS)+3*j,0,len(OUTPUT_FILE_HEADERS)+3*j+2,k)
        comment_key_to_index[k] = len(OUTPUT_FILE_HEADERS)+3*j
        j+=1
    i = 1
    for k,v in file_map.items():
        comment_index = comment_key_to_index[k]
        for people_response in v:
            email = people_response[5]
            if email not in PEOPLE_DETAILS:
                print("EMAIL NOT IN RECORDS",email)
                continue
            people_data = PEOPLE_DETAILS[email]

            ws.write(i,0,email)
            ws.write(i,1,people_data[1])
            ws.write(i,2,people_data[2])
            ws.write(i,3,people_data[5])
            ws.write(i,4,people_data[-4])
            ws.write(i,comment_index,people_response[2])
            ws.write(i,comment_index+1,people_response[3])
            if people_response[4] != people_response[4]:
                people_response[4] = ""
            ws.write(i,comment_index+2,people_response[4])

            i += 1
    workbook.close()

EMAIL NOT IN RECORDS arunjose36@gmail.com
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT IN RECORDS amiya.pradhan@capgemini.com
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT IN RECORDS amiya.pradhan@capgemini.com
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT IN RECORDS priyamd@iitkgp.ac.in
EMAIL NOT IN RECORDS iyermohan@iisc.ac.in
EMAIL NOT IN RECORDS amiya.pradhan@capgemini.com
EMAIL NOT IN RECORDS sg@iitkgp.ac.in
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT IN RECORDS priyamd@iitkgp.ac.in
EMAIL NOT IN RECORDS amiya.pradhan@capgemini.com
EMAIL NOT IN RECORDS sg@iitkgp.ac.in
EMAIL NOT IN RECORDS swagatikasahoos@gmail.com
EMAIL NOT IN RECORDS pkpati@ddn.com
EMAIL NOT I